In [32]:
# import necessary libraries
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor

# set llm model
model = ChatOpenAI(model="gpt-4o", temperature=0)

In [33]:
#load agent tools
from agent_tools.portfolio_manager_tools import place_order, get_portfolio_state

portfolio_manager_agent = create_react_agent(
    model=model,
    tools=[place_order, get_portfolio_state],
    name="portfolio_manager",
    prompt="""
    You are a portfolio manager. Your task is to 
    inform the user about the state of their portfolio 
    and buy or sell stocks. Always use one tool at a time."""
)

In [34]:
from agent_tools.financial_analyst_tools import get_analyst_price_targets, get_recommendations
from agent_tools.financial_analyst_tools import get_fundamental_analysis, get_technical_analysis


financial_analyst_agent = create_react_agent(

    model=model,
    tools=[
        get_analyst_price_targets,  
        get_recommendations,  
        get_fundamental_analysis,
        get_technical_analysis
    ],
    name="financial_analyst",
    prompt="""
You are a financial analyst agent with expertise in fundamental, technical, and sentiment analysis. Your role is to deliver actionable, data-driven insights on companies.
Capabilities:
1. Fundamental Analysis: Evaluate financial health, business model, management quality, key ratios, and growth potential.
2. Technical Analysis: Analyze price trends, trading volume, support/resistance levels, and technical indicators for short- and medium-term outlooks.
3. Social Sentiment Analysis: Monitor real-time social media sentiment, identify trending topics, and gauge public perception.
4. Competitive Analysis: Assess key competitors, compare market share, evaluate industry positioning, and analyze performance metrics.
Stay objective, current, and focused on delivering insightful financial analysis.
"""
)

In [35]:
from agent_tools.company_research_tools import get_basic_info, get_quarterly_report
from agent_tools.company_research_tools import search_financial_news, get_balance_sheet


research_agent = create_react_agent(

    model=model,
    tools=[
        get_basic_info,  
        get_quarterly_report,  
        search_financial_news,
        get_balance_sheet
    ],
    name="company_research",
    prompt="""
You are a company research agent specializing in gathering and analyzing fundamental company information. Your role is to provide comprehensive insights about companies based on available data.

Your capabilities include:
1. Basic Company Information: Retrieve and analyze company overview, key statistics, and general business information.
2. Financial Reports: Access and interpret quarterly financial reports to understand company performance.
3. Financial News: Search and analyze relevant financial news to provide context about company developments.
4. Balance Sheet Analysis: Examine and interpret company balance sheets to assess financial health.

"""
)


In [36]:
# Create supervisor workflow
workflow = create_supervisor(
    [portfolio_manager_agent, research_agent, financial_analyst_agent],
    model=model,
    prompt=(
        """You are the Supervisor Agent in a multi-agent financial assistant system. Your role is to interpret the user's prompt and delegate it to the most appropriate specialized agent. There are currently two available agents:

1. **PortfolioManagerAgent**
   - Handles all questions and commands related to the user's investment portfolio.
   - Capable of informing the user about the state of the portfolio.
   - Can perform **buy** and **sell** operations.

2. **ResearchAgent**
   - Provides comprehensive company information.
   - Can retrieve basic company information, quarterly reports, financial news, and balance sheets.
   - Focuses on fundamental company data and financial reporting.

3. **FinancialAnalystAgent**
   - Specializes in financial analysis and market insights.
   - Provides analyst price targets, recommendations, technical and fundamental analysis.
   - Offers expert analysis of company performance and market trends.

Your job is to:
- Understand the user's intent.
- Route the prompt to the correct agent.
- If the request is ambiguous, ask clarifying questions to better understand what the user wants.
- If the task doesn't match either agent's capabilities, politely inform the user and suggest an alternative or ask for more details.

Always be helpful, concise, and precise in your routing.
"""
    )
)

# Compile and run
graph = workflow.compile()


In [40]:
from IPython.display import Image, display


try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except:
    graph.get_graph().print_ascii()





                                                 +-----------+                                              
                                                 | __start__ |                                              
                                                 +-----------+                                              
                                                       *                                                    
                                                       *                                                    
                                                       *                                                    
                                                +------------+                                              
                                             ***| supervisor |.....                                         
                                     ********   +------------+*    ........                                 
                   

# Testing Portfolio Manager Agent

In [ ]:
result = graph.invoke({
    "messages": [
        {
            "role": "user",
            "content": "What is the current state of my portfolio?"
        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

What is the current state of my portfolio?
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_portfolio_manager (call_hB23YzbB6Ly8EOcYOwpatrMT)
 Call ID: call_hB23YzbB6Ly8EOcYOwpatrMT
  Args:
================================= Tool Message =================================
Name: transfer_to_portfolio_manager

Successfully transferred to portfolio_manager
================================== Ai Message ==================================
Name: portfolio_manager

Here's the current state of your portfolio:

- **Cash:** $55,987.73
- **Buying Power:** $167,924.96
- **Portfolio Value:** $111,937.23
- **Equity:** $111,937.23

### Positions:
1. **Microsoft (MSFT)**
   - Quantity: 50 shares
   - Average Entry Price: $352.41
   - Current Price: $437.33
   - Market Value: $21,866.50
   - Unrealized Profit/Loss: $4,245.86 (24.10%)

2. **NVIDIA (NVD

# Testing Research Agent

In [42]:
result = graph.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Please provide me with the latest financial news about Pfizer"
        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

Please provide me with the latest financial news about Pfizer
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_company_research (call_6cLdUuqxQTB4AcsNVpbGMkDA)
 Call ID: call_6cLdUuqxQTB4AcsNVpbGMkDA
  Args:
================================= Tool Message =================================
Name: transfer_to_company_research

Successfully transferred to company_research
================================== Ai Message ==================================
Name: company_research

Here are the latest financial news headlines about Pfizer:

1. **Pfizer (PFE) Laps the Stock Market**: Pfizer closed the most recent trading day at $22.15, indicating some movement in the stock market.

2. **Pfizer Inc. (PFE) Stock Price, News, Quote & History**: The company is part of the healthcare sector, specifically in the drug manufacturers - general industry.

# Test Financial Analyst

In [45]:
result = graph.invoke({
    "messages": [
        {
            "role": "user",
            # "content": "Give me the fundamental analysis of Google"
            "content": "Give me analyst price targets for Google"

        }
    ]
})

for m in result["messages"]:
    m.pretty_print()

================================ Human Message =================================

Give me analyst price targets for Google
================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_financial_analyst (call_XAjRcrw9hzdGRfjqU66ORd3s)
 Call ID: call_XAjRcrw9hzdGRfjqU66ORd3s
  Args:
================================= Tool Message =================================
Name: transfer_to_financial_analyst

Successfully transferred to financial_analyst
================================== Ai Message ==================================
Name: financial_analyst

It seems that there's a temporary issue with retrieving the analyst price targets for Google (GOOGL) due to rate limits. Please try again later, or if you have any other questions or need analysis on a different aspect, feel free to ask!
================================== Ai Message ==================================
Name: financial_analyst

Transferring back to supervisor
Tool Call